Data Source: https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

Features consist of hourly average ambient variables

Temperature (T) in the range 1.81°C and 37.11°C,
Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
Relative Humidity (RH) in the range 25.56% to 100.16%
Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
Net hourly electrical energy output (EP) 420.26-495.76 MW
The averages are taken from various sensors located around the plant that record the ambient variables every second. The variables are given without normalization.

Dataset Information:

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP) of the plant. 
A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum is colected from and has effect on the Steam Turbine, he other three of the ambient variables effect the GT performance.

In [1]:
!ls -ltr /data

ls: cannot access '/data': No such file or directory


In [2]:
import os, sys
SPARK_HOME = os.environ['SPARK_HOME']
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "lib", "py4j-0.10.4-src.zip"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/10/29 17:37:50 WARN Utils: Your hostname, long-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/29 17:37:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/29 17:37:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

# Load Data

In [4]:
print('Long: 22022604')

df = spark.read.format("csv").option("header","true")\
.option("inferSchema","true").load("./data/Combined_Cycle_Power_Plant.csv")

Long: 22022604


In [5]:
print('Long: 22022604')
df.show()

Long: 22022604
+---+-----+-----+-------+-----+------+
|_c0|   AT|    V|     AP|   RH|    PE|
+---+-----+-----+-------+-----+------+
|  0|14.96|41.76|1024.07|73.17|463.26|
|  1|25.18|62.96|1020.04|59.08|444.37|
|  2| 5.11| 39.4|1012.16|92.14|488.56|
|  3|20.86|57.32|1010.24|76.64|446.48|
|  4|10.82| 37.5|1009.23|96.62| 473.9|
|  5|26.27|59.44|1012.23|58.77|443.67|
|  6|15.89|43.96|1014.02|75.24|467.35|
|  7| 9.48|44.71|1019.12|66.43|478.42|
|  8|14.64| 45.0|1021.78|41.25|475.98|
|  9|11.74|43.56|1015.14|70.72| 477.5|
| 10|17.99|43.72|1008.64|75.04|453.02|
| 11|20.14|46.93|1014.66|64.22|453.99|
| 12|24.34| 73.5|1011.31|84.15|440.29|
| 13|25.71|58.59|1012.77|61.83|451.28|
| 14|26.19|69.34|1009.48|87.59|433.99|
| 15|21.42|43.79|1015.76|43.08|462.19|
| 16|18.21| 45.0|1022.86|48.84|467.54|
| 17|11.04|41.74| 1022.6|77.51| 477.2|
| 18|14.45|52.75|1023.97|63.59|459.85|
| 19|13.97|38.47|1015.15|55.28| 464.3|
+---+-----+-----+-------+-----+------+
only showing top 20 rows



24/10/29 17:38:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , AT, V, AP, RH, PE
 Schema: _c0, AT, V, AP, RH, PE
Expected: _c0 but found: 
CSV file: file:///home/hadoop/Documents/UET-BigData/Lab08/data/Combined_Cycle_Power_Plant.csv


In [6]:
print('Long: 22022604')
df.cache()

Long: 22022604


DataFrame[_c0: int, AT: double, V: double, AP: double, RH: double, PE: double]

# Convert Spark Dataframe to Pandas Dataframe

In [7]:
print('Long: 22022604')
df.limit(10).toPandas().head() 

Long: 22022604


24/10/29 17:38:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , AT, V, AP, RH, PE
 Schema: _c0, AT, V, AP, RH, PE
Expected: _c0 but found: 
CSV file: file:///home/hadoop/Documents/UET-BigData/Lab08/data/Combined_Cycle_Power_Plant.csv


,_c0,AT,V,AP,RH,PE
0,0,14.96,41.76,1024.07,73.17,463.26
1,1,25.18,62.96,1020.04,59.08,444.37
2,2,5.11,39.40,1012.16,92.14,488.56
3,3,20.86,57.32,1010.24,76.64,446.48
4,4,10.82,37.50,1009.23,96.62,473.90


## Verctorize the features

In [8]:
print('Long: 22022604')
from pyspark.ml.feature import *

Long: 22022604


In [9]:
print('Long: 22022604')
vectorizer = VectorAssembler()
vectorizer.setInputCols(["AT", "V", "AP", "RH"])
vectorizer.setOutputCol("features")

df_vect = vectorizer.transform(df)
df_vect.show(10, False)

Long: 22022604
+---+-----+-----+-------+-----+------+---------------------------+
|_c0|AT   |V    |AP     |RH   |PE    |features                   |
+---+-----+-----+-------+-----+------+---------------------------+
|0  |14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024.07,73.17]|
|1  |25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020.04,59.08]|
|2  |5.11 |39.4 |1012.16|92.14|488.56|[5.11,39.4,1012.16,92.14]  |
|3  |20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010.24,76.64]|
|4  |10.82|37.5 |1009.23|96.62|473.9 |[10.82,37.5,1009.23,96.62] |
|5  |26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012.23,58.77]|
|6  |15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014.02,75.24]|
|7  |9.48 |44.71|1019.12|66.43|478.42|[9.48,44.71,1019.12,66.43] |
|8  |14.64|45.0 |1021.78|41.25|475.98|[14.64,45.0,1021.78,41.25] |
|9  |11.74|43.56|1015.14|70.72|477.5 |[11.74,43.56,1015.14,70.72]|
+---+-----+-----+-------+-----+------+---------------------------+
only showing top 10 rows



In [10]:
print('Long: 22022604')
print(vectorizer.explainParams())

Long: 22022604
handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)
inputCols: input column names. (current: ['AT', 'V', 'AP', 'RH'])
outputCol: output column name. (default: VectorAssembler_e7b30eed2dc1__output, current: features)


## Fit Linear Regression Model

In [11]:
from pyspark.ml.regression import LinearRegression

In [12]:
print('Long: 22022604')
lr = LinearRegression()
print(lr.explainParams())

Long: 22022604
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (defau

In [14]:
print('Long: 22022604')
lr.setLabelCol("PE")
lr.setFeaturesCol("features")
model = lr.fit(df_vect)

Long: 22022604


24/10/29 17:40:02 WARN Instrumentation: [ab93e212] regParam is zero, which might cause numerical instability and overfitting.


In [15]:
type(model)

pyspark.ml.regression.LinearRegressionModel

### View model summary

In [16]:
print('Long: 22022604')
print("R2:", model.summary.r2)
print("Intercept: ", model.intercept, "Coefficients", model.coefficients)

Long: 22022604
R2: 0.9286960898122542
Intercept:  454.6092744252279 Coefficients [-1.9775131067205514,-0.23391642256571113,0.06208294367428365,-0.1580541029323118]


### Predict

In [17]:
print('Long: 22022604')
df_pred = model.transform(df_vect)
df_pred.show()

Long: 22022604
+---+-----+-----+-------+-----+------+--------------------+------------------+
|_c0|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+---+-----+-----+-------+-----+------+--------------------+------------------+
|  0|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...|467.26978995931074|
|  1|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.07736589754256|
|  2| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...| 483.5626426259766|
|  3|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...|450.55566824634656|
|  4|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....| 471.8254985833747|
|  5|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...| 442.3093914004613|
|  6|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|463.96498106341966|
|  7| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....| 478.1744824201466|
|  8|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....| 472.0476219689337|
|  9|11.74|43.56|1015.14|70.72| 477.5

### Evaluate

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

In [19]:
print('Long: 22022604')
evaluator = RegressionEvaluator()
print(evaluator.explainParams())

Long: 22022604
labelCol: label column name. (default: label)
metricName: metric name in evaluation - one of:
                       rmse - root mean squared error (default)
                       mse - mean squared error
                       r2 - r^2 metric
                       mae - mean absolute error
                       var - explained variance. (default: rmse)
predictionCol: prediction column name. (default: prediction)
throughOrigin: whether the regression is through the origin. (default: False)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)


In [21]:
print('Long: 22022604')
evaluator = RegressionEvaluator(labelCol = "PE", 
                                predictionCol = "prediction", 
                                metricName = "rmse")
evaluator.evaluate(df_pred)

Long: 22022604


4.557126016749477

## Build a pipeline

In [22]:
from pyspark.ml.pipeline import Pipeline, PipelineModel

In [23]:
print('Long: 22022604')
pipeline = Pipeline()
print(pipeline.explainParams())
pipeline.setStages([vectorizer, lr])
pipelineModel = pipeline.fit(df)

Long: 22022604
stages: a list of pipeline stages (undefined)


24/10/29 17:40:23 WARN Instrumentation: [a3445ac9] regParam is zero, which might cause numerical instability and overfitting.


In [24]:
print('Long: 22022604')
pipeline.getStages()

Long: 22022604


[VectorAssembler_e7b30eed2dc1, LinearRegression_fb87c90d60e5]

In [25]:
print('Long: 22022604')
lr_model = pipelineModel.stages[1]
lr_model .coefficients

Long: 22022604


DenseVector([-1.9775, -0.2339, 0.0621, -0.1581])

In [26]:
print('Long: 22022604')
pipelineModel.transform(df).show()

Long: 22022604


+---+-----+-----+-------+-----+------+--------------------+------------------+
|_c0|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+---+-----+-----+-------+-----+------+--------------------+------------------+
|  0|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...|467.26978995931074|
|  1|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.07736589754256|
|  2| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...| 483.5626426259766|
|  3|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...|450.55566824634656|
|  4|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....| 471.8254985833747|
|  5|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...| 442.3093914004613|
|  6|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|463.96498106341966|
|  7| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....| 478.1744824201466|
|  8|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....| 472.0476219689337|
|  9|11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1

In [27]:
print('Long: 22022604')
evaluator.evaluate(pipelineModel.transform(df))

Long: 22022604


4.557126016749477

## Save the pipeline to disk to persist the model

In [28]:
print('Long: 22022604')
pipelineModel.save("/tmp/lr-pipeline")

Long: 22022604


In [29]:
print('Long: 22022604')
!tree /tmp/lr-pipeline

Long: 22022604
/bin/bash: line 1: tree: command not found


### Load the persisted model from the disk

In [30]:
print('Long: 22022604')
saved_model = PipelineModel.load("/tmp/lr-pipeline")
saved_model.stages[1].coefficients

Long: 22022604


DenseVector([-1.9775, -0.2339, 0.0621, -0.1581])

In [31]:
print('Long: 22022604')
saved_model.transform(df).show()

Long: 22022604
+---+-----+-----+-------+-----+------+--------------------+------------------+
|_c0|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+---+-----+-----+-------+-----+------+--------------------+------------------+
|  0|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...|467.26978995931074|
|  1|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.07736589754256|
|  2| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...| 483.5626426259766|
|  3|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...|450.55566824634656|
|  4|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....| 471.8254985833747|
|  5|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...| 442.3093914004613|
|  6|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|463.96498106341966|
|  7| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....| 478.1744824201466|
|  8|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....| 472.0476219689337|
|  9|11.74|43.56|1015.14|70.72| 477.5

In [32]:
print('Long: 22022604')
df_train, df_test = df.randomSplit(weights=[0.7, 0.3], seed = 200)

Long: 22022604


In [33]:
print('Long: 22022604')
pipelineModel = pipeline.fit(df_train)
evaluator.evaluate(pipelineModel.transform(df_test))

Long: 22022604


24/10/29 17:40:34 WARN Instrumentation: [ad719a09] regParam is zero, which might cause numerical instability and overfitting.


4.549676410555844

# Tune the model

In [34]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [35]:
print('Long: 22022604')
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           trainRatio=0.8)

tuned_model = tvs.fit(vectorizer.transform(df_train))

Long: 22022604


In [36]:
print('Long: 22022604')
tuned_model.bestModel, tuned_model.validationMetrics

Long: 22022604


(LinearRegressionModel: uid=LinearRegression_fb87c90d60e5, numFeatures=4,
 [4.9307586862321875,
  5.321834156404265,
  35.05740790149438,
  4.4693431517698246,
  4.469889338576265,
  4.472125730700819,
  4.930967152884744,
  5.154731052956273,
  5.171572070740107,
  4.468010254274359,
  4.4680856548135095,
  4.468179339387101])

In [37]:
print('Long: 22022604')
df_test_pred = tuned_model.transform(vectorizer.transform(df_test))
df_test_pred.show()

Long: 22022604
+---+-----+-----+-------+-----+------+--------------------+------------------+
|_c0|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+---+-----+-----+-------+-----+------+--------------------+------------------+
|  1|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.15315979981034|
|  4|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....|471.77109083226196|
|  9|11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1015...|473.03037563407264|
| 14|26.19|69.34|1009.48|87.59|433.99|[26.19,69.34,1009...|435.43459020882375|
| 20|17.76|42.42|1009.09|66.26|468.27|[17.76,42.42,1009...|461.72562067563223|
| 24|27.36| 48.6|1003.18|54.93|436.06|[27.36,48.6,1003....| 442.7390702892602|
| 27| 7.91|39.96|1023.57|88.44|475.52|[7.91,39.96,1023....| 479.1996652620346|
| 30|23.87|63.94|1019.02|44.28|445.11|[23.87,63.94,1019...| 448.7746732176651|
| 34|24.81|63.94|1018.76|44.51|444.26|[24.81,63.94,1018...| 446.8678874634908|
| 36|24.66|63.73| 1011.4|74.52|444.37

In [38]:
print('Long: 22022604')
evaluator.evaluate(df_test_pred)

Long: 22022604


4.54972684185712